In [ ]:
import pandas as pd
import datetime as dt

# Contact Gaps

In [ ]:
chs = pd.read_stata(r"C:\Users\jc4673\Documents\CHS_Lexis\contactsbyalt_key.dta")
date_death = pd.read_csv(r"C:\Users\jc4673\Documents\CHS_Lexis\LexisNexis\LN_NOID_DATA\original" +
    r"\LN_Output_Voter_LN_InputLexisNexisCHSParticipantsNS.Dataset.csv", usecols=['ssn_altkey', 'yrdeath'])
del chs['ttolastcontact']

# Join in Contact Dates from Addresses

In [ ]:
chs_address = pd.read_csv(r"C:\Users\jc4673\Documents\CHS_Lexis\distance_measures\chs_lexisnexis_distance_measures_2\
\chs_addresses_geocode__chs_addresses_geocode.csv", usecols=['ssn_altkey', 'dxid1', 'dxid2','chs_entrydate1', 'chs_entrydate2'])

In [ ]:
start_date = chs_address.groupby("ssn_altkey").first().reset_index(drop=False)
merged = chs.merge(start_date[['ssn_altkey', 'chs_entrydate1']], on=['ssn_altkey'], how='inner')
merged['chs_entrydate1'] = pd.to_datetime(merged['chs_entrydate1'])

In [ ]:
merged['ttocontact'] = pd.to_timedelta(merged['ttocontact'], unit='D')
merged['contact_date'] = merged['chs_entrydate1'] + merged['ttocontact']
del merged['chs_entrydate1']

In [ ]:
merged['gap_before'] = merged.ttocontact - merged.groupby('ssn_altkey').ttocontact.shift(1)

In [ ]:
merged

# Create Address Table

In [ ]:
chs_address_1 = chs_address[['ssn_altkey', 'dxid1', 'chs_entrydate1']]
chs_address_2 = chs_address[['ssn_altkey', 'dxid2', 'chs_entrydate2']].dropna(how='any')

In [ ]:
chs_address_1 = chs_address_1.drop_duplicates().rename(columns={"dxid1": "dxid", "chs_entrydate1": "chs_entrydate"}).set_index(['ssn_altkey', 'dxid'])
chs_address_2 = chs_address_2.drop_duplicates().rename(columns={"dxid2": "dxid", "chs_entrydate2": "chs_entrydate"}).set_index(['ssn_altkey', 'dxid'])

In [ ]:
address_dates = chs_address_1.combine_first(chs_address_2).reset_index(drop=False)

In [ ]:
address_dates.reset_index(inplace=True)
del address_dates['dxid']
address_dates['index'] = address_dates.groupby('ssn_altkey').cumcount()
address_dates['chs_entrydate'] = pd.to_datetime(address_dates['chs_entrydate'])

In [ ]:
address_dates_wide = address_dates.pivot(index='ssn_altkey', columns='index', values='chs_entrydate')